In [ ]:
# Importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader



In [ ]:
# Setting device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading the CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
cifar10_data = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
dataloader = DataLoader(cifar10_data, batch_size=128, shuffle=True)



In [ ]:
# Generator class
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=3):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.gen(x)



In [ ]:
# Discriminator class
class Discriminator(nn.Module):
    def __init__(self, img_channels=3):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(img_channels, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.disc(x)


In [ ]:
# Initialize Generator and Discriminator
z_dim = 100
img_channels = 3
gen = Generator(z_dim, img_channels).to(device)
disc = Discriminator(img_channels).to(device)

# Optimizers and Loss function
lr = 0.0002
gen_opt = optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
disc_opt = optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))
criterion = nn.BCELoss()

In [ ]:
# Training the DCGAN
epochs = 2
fixed_noise = torch.randn(25, z_dim, 1, 1).to(device)
g_losses, d_losses = [], []

for epoch in range(epochs):
    g_loss_epoch, d_loss_epoch = 0, 0
    for real, _ in dataloader:
        real = real.to(device)
        batch_size = real.size(0)

        # Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        noise = torch.randn(batch_size, z_dim, 1, 1).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        disc_fake = disc(fake.detach()).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2

        disc_opt.zero_grad()
        lossD.backward()
        disc_opt.step()

        # Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z)))
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))

        gen_opt.zero_grad()
        lossG.backward()
        gen_opt.step()

        g_loss_epoch += lossG.item()
        d_loss_epoch += lossD.item()

    # Save losses
    g_losses.append(g_loss_epoch / len(dataloader))
    d_losses.append(d_loss_epoch / len(dataloader))

    # Visualize progress every 10 epochs
    if epoch % 10 == 0 or epoch == epochs - 1:
        with torch.no_grad():
            fake_images = gen(fixed_noise).cpu()
            grid = torchvision.utils.make_grid(fake_images, nrow=5, normalize=True)
            plt.figure(figsize=(10, 10))
            plt.imshow(np.transpose(grid, (1, 2, 0)))
            plt.title(f"Epoch {epoch}")
            plt.axis("off")
            plt.show()

    print(f"Epoch [{epoch+1}/{epochs}] | Loss D: {d_losses[-1]:.4f}, Loss G: {g_losses[-1]:.4f}")



In [ ]:
# Plotting Loss Curves
plt.figure(figsize=(10, 5))
plt.plot(g_losses, label="Generator Loss")
plt.plot(d_losses, label="Discriminator Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Generator and Discriminator Loss Curves")
plt.show()

# Visualize Generated Images
with torch.no_grad():
    fake_images = gen(fixed_noise).cpu()
    grid = torchvision.utils.make_grid(fake_images, nrow=5, normalize=True)
    plt.figure(figsize=(10, 10))
    plt.imshow(np.transpose(grid, (1, 2, 0)))
    plt.title("Generated Images after Training")
    plt.axis("off")
    plt.show()



In [ ]:
# Visualize Real CIFAR-10 Images
real_images, _ = next(iter(dataloader))
real_images = real_images[:25]
real_grid = torchvision.utils.make_grid(real_images, nrow=5, normalize=True)
plt.figure(figsize=(10, 10))
plt.imshow(np.transpose(real_grid, (1, 2, 0)))
plt.title("Real CIFAR-10 Images")
plt.axis("off")
plt.show()

# Comment on Generated Images
print("The generated images resemble the CIFAR-10 dataset, showcasing the ability of the DCGAN to learn complex image structures."
      " While some generated images are more realistic, there is still variability in quality, indicating areas for further training or model adjustment.")
